## Propositional variation with MFT Test

In [25]:
#this cell of code you can comment out once you installed the following tools in your environment
# pip install panda
# pip install allennlp-models

In [40]:
# from sklearn.metrics import classification_report, confusion_matrix
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from allennlp_models import pretrained
import pandas as pd
import json
import ast
import csv
import os
os.chdir(r'C:\Users\anaverageone\htlt_env\Advanced_NLP')

In [41]:
# Loading BERT_based and BiLSTM models from AllenNLP tool package
Bert_based = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")
BiLSTM = pretrained.load_predictor("structured-prediction-srl")

error loading _jsonnet (this is expected on Windows), treating C:\Users\ANAVER~1\AppData\Local\Temp\tmp0er_6lmj\config.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json
error loading _jsonnet (this is expected on Windows)

In [44]:
# create original data set in the format of lists
# sentence samples are originally from en_ewt-up-train.conllu & en_ewt-up-test.conllu
sentences = [
    "Along with an area on the page for those to join the mailing list.", 
    "I went with Natasha.",
    "Deep tissue massage helps with pain in neck and shoulders",
    "The setting feels like a Sushi bar in NYC; small, cozy, but with flair.",
    "They have awesome service with a smile :)"]

gold = [
   ["O", "O", "O", "O", "O", "O", "O", "O", "B-ARG0", "O", "B-V", "O", "O", "B-ARG1", "O"],
    ["B-ARG0", "B-V", "O","B-ARG1" "O"] ,
    ["O", "O", "B-ARG0", "B-V", "O", "B-ARG1", "O", "O", "O", "O"] ,
    ["O", "B-ARG1", "B-V", "O", "O", "O", "B-ARG2", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"],
    ["B-ARG0", "B-V", "B-ARGM-ADJ","B-ARG1", "O", "O", "B-ARGM-MNR", "O"]]

In [45]:
# a function that saves challenge data as csv file 
def create_challenge_data(outfile, test_samples, test_gold):
    with open(outfile, mode='w', newline='') as file:
        writer = csv.writer(file)
    # write headers
        writer.writerow(["Sentence", "Gold"])
    # write each sentence and its corresponding gold labels
        for i in range(len(test_samples)):
            writer.writerow([test_samples[i], test_gold[i]])
            
# a function that prints out the failure rate of model on corresponding challenge data set 
def get_Pred_results(outfile, predictor_name, test_samples):
    with open(outfile, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
    # write headers
        writer.writerow(["Sentence", "Arguments"])
    # Loop through sentences and arguements
        for sent in test_samples:
            output = predictor_name.predict(sentence=sent)
        # for sentences with more than one predicate
            if len(output["verbs"]) > 1:
                arguments = output["verbs"][1]["tags"]
                print(arguments)
                writer.writerow([sent, arguments])
        # for sentences with one predicte 
            elif len(output["verbs"]) == 1:
                arguments = output["verbs"][0]["tags"]
                print(arguments)
                writer.writerow([sent, arguments])
        # for sentences without verbs
            else:
                writer.writerow([sent, " "])
                print('NaN')

# a function that iterate through each token and find matches(both span & lable count as correct)
def calculate_failure_rate(model_name, test_name, gold_arg, model_arg):
    matches_counter = 0
    token_counter = 0
    for i, sent_arg in enumerate(gold_arg):
        gold_list = ast.literal_eval(sent_arg)
        pred_list = ast.literal_eval(model_arg[i])
        for i in range(len(gold_list)):
            if gold_list[i] != pred_list[i]:
                token_counter += 1
            else:
                token_counter += 1
                matches_counter += 1
    failure_rate = (token_counter - matches_counter) / token_counter
    print(f'{model_name} model yeilds a {failure_rate} failure rate for {test_name} test!')

In [46]:
# call "create_challenge_data" to save challenge data as csv file(s)
create_challenge_data('data/proposition_file.csv', sentences, gold)

In [47]:
# call "get_Pred_results" to save model predictions as csv file(s)
get_Pred_results('data/Bertbased_proposition.csv', Bert_based, sentences)
print()
get_Pred_results('data/BiLSTM_proposition.csv', BiLSTM, sentences)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O']
['B-ARG0', 'B-V', 'B-ARGM-COM', 'I-ARGM-COM', 'O']
['B-ARG0', 'I-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1']
['B-ARG1', 'I-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR', 'I-ARGM-MNR', 'O']

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'O']
['B-ARG0', 'B-V', 'B-ARGM-COM', 'I-ARGM-COM', 'O']
['B-ARG0', 'I-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1']
['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']
['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG

### Evaluation (failure rate) on challenge data (token level)

In [48]:
# Read the test data file and model predictions as pandas dataframe
df_gold = pd.read_csv('data/proposition_file.csv', sep=',')
df_bert = pd.read_csv('data/Bertbased_proposition.csv', sep=',')
df_bilstm = pd.read_csv('data/BiLSTM_proposition.csv', sep=',')
df_gold.fillna(0)
df_bert.fillna(0)
df_bilstm.fillna(0)
# prepare argument labels for comparison 
bert_arg = df_bert['Arguments'].tolist()
bilstm_arg = df_bilstm['Arguments'].tolist()
gold_arg = df_gold['Gold'].tolist()

### Bert-based model

In [49]:
# call the caculation function to compute failure rate for each model
calculate_failure_rate('Bertbased', 'Propositional construction', gold_arg, bert_arg)

Bertbased model yeilds a 0.6363636363636364 failure rate for Propositional construction test!


### BiLSTM 

In [50]:
calculate_failure_rate('BiLSTM', 'Propositional construction', gold_arg, bilstm_arg)

BiLSTM model yeilds a 0.6545454545454545 failure rate for Propositional construction test!
